<a href="https://colab.research.google.com/github/isabella-001/photofolio-app/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets wandb pyyaml torch

In [3]:
# Log in to Weights & Biases (Optional but recommended for tracking)
import wandb
wandb.login()


True

In [4]:
# Verify TPU/GPU availability
import torch
print(f"Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

Device: cuda


In [6]:
!pip install transformers "datasets<3.0.0" wandb pyyaml torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.


In [7]:
# Log in to Weights & Biases (Optional but recommended for tracking)
import wandb
wandb.login()

True

In [8]:
# Verify TPU/GPU availability
import torch
print(f"Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

Device: cuda


In [9]:
# Download the WikiSQL Dataset
# This script handles downloading through the `datasets` library.
!python download_data.py

Checking dependencies...
Generating test split: 100% 15878/15878 [00:03<00:00, 4725.68 examples/s]
Generating validation split: 100% 8421/8421 [00:01<00:00, 5035.34 examples/s]
Generating train split: 100% 56355/56355 [00:13<00:00, 4067.55 examples/s]
Processing train -> MyTrials/train.json...
100% 56355/56355 [00:24<00:00, 2307.07it/s]
Processing validation -> MyTrials/validation.json...
100% 8421/8421 [00:03<00:00, 2517.75it/s]
Processing test -> MyTrials/test.json...
100% 15878/15878 [00:07<00:00, 2157.85it/s]
Done! Data saved to MyTrials


In [10]:
# Start Training
# This runs the training loop defined in train.py
!python train.py


train data path: /content/MyTrials/train.json
tokenizer_config.json: 100% 2.32k/2.32k [00:00<00:00, 14.9MB/s]
spiece.model: 100% 792k/792k [00:00<00:00, 4.77MB/s]
tokenizer.json: 100% 1.39M/1.39M [00:00<00:00, 5.66MB/s]
Creating new model...
wandb: Currently logged in as: tahasanhabib83 (tahasanhabib83-rajshahi-university-of-engineering-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ Waiting for wandb.init()...
wandb: ⣽ setting up run 8v0l0188 (0.2s)
wandb: ⣾ setting up run 8v0l0188 (0.2s)
wandb: Tracking run with wandb version 0.23.1
wandb: Run data is saved locally in /content/wandb/run-20251214_135359-8v0l0188
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run good-meadow-1
wandb: ⭐️ View project at https://wandb.ai/tahasanhabib83-rajshahi-university-of-engineering-technology/transformer-sql-fixed
wandb: 🚀 View run at https://wandb.ai/tahasanhabib83-rajshahi-university-of-engineering-tech

In [16]:
import json
import random
import os
from inference import SqlInference

def load_random_sample(filepath):
    if not os.path.exists(filepath):
        print(f"Error: Data file not found at {filepath}")
        return None

    with open(filepath, 'r') as f:
        lines = f.readlines()
        if not lines:
            return None
        return json.loads(random.choice(lines))

def main():
    # Initialize the inference class
    infer = SqlInference()

    # Path to your training data
    # Adjust this if your data is in a different location
    data_path = "MyTrials/train.json"

    print(f"Attempting to load sample from {data_path}...")
    sample = load_random_sample(data_path)

    if sample:
        question = sample['question']
        table = sample['table']
        human_readable_sql = sample.get('sql', {}).get('human_readable', 'N/A')

        print("\n" + "="*50)
        print("SAMPLE FROM DATASET")
        print("="*50)
        print(f"Question: {question}")
        print(f"Expected SQL: {human_readable_sql}")
        print("-" * 50)

        # Generate prediction
        generated_sql = infer.predict(question, table)

        print(f"Generated SQL: {generated_sql}")
        print("="*50 + "\n")

    else:
        # Fallback to manual example if file not found
        print("Could not load data file. Using manual example instead.")
        question = "How many players are from United States"
        table = {
            "header": ["Player", "Nationality"],
            "rows": [
                ["Shawn Respert", "United States"],
                ["Aleksandar Radojevi", "Serbia"],
                ["Quentin Richardson", "United States"]
            ]
        }
        print("-" * 50)
        print(f"Question: {question}")
        print("-" * 50)
        generated_sql = infer.predict(question, table)
        print(f"Generated SQL: {generated_sql}")
        print("-" * 50)

if __name__ == "__main__":
    main()


model loaded from /content/transformer_model.pt
Attempting to load sample from MyTrials/train.json...

SAMPLE FROM DATASET
Question: what's the result with record being 0–1
Expected SQL: SELECT Result FROM table WHERE Record = 0–1
--------------------------------------------------
Encoder input: The question is: what's the result with record being 0–1 table: Week | Date | Opponent | Result | Record | Game Site | Attendance
1 | September 13, 1964 | at Minnesota Vikings | L 24–34 | 0–1 | Metrop...
Generated SQL: SELECT Result FROM table WHERE Record = L 4–1



In [ ]:
from google.colab import drive
drive.mount('/content/drive')